# 1. Introducción

En el siguiente ejercicio se realizará el **cálculo aproximado de la famosa cifra $\pi$** a partir de la serie que da la solución al famoso problema matemático llamado "El Problema de Basilea":

<center>$\sum_{n=1}^{\infty}\frac{1}{n^2}=\frac{\pi^2}{6}$</center>

El algoritmo está basado en la idea de que, a medida que sumemos más símbolos de la serie, más preciso será el valor calculado de $\pi$ a la hora de despejar la ecuación [1].

La idea principal es mostrar la perfomance de un algoritmo que puede ser paralelizado con respecto a su resolución secuencial, para ello, implementaremos la interfaz **OpenMP** [2].

En este aspecto, se implementó la directiva del compilador **#pragma omp parallel for reduction (...)** [3] para que se paralelice la sección del código donde se aplica el algoritmo. Cada ciclo del for será atendido por hilo. También se aplica la directiva de reducción para que, una vez finalizado cada hilo, se aplique a la variable compartida el valor resultante de la operación a través de una operación especificada (en nuestro caso, una suma).

Si bien se utiliza Python [4] para creación del archivo, el programa en sí está desarrollado en C++, donde su compilación y ejecución se realiza a través de comandos tipo UNIX en Colab [5][6].

---
# 2. Armado del ambiente


### Importación de **bibliotecas**

In [ ]:
import pandas as pd # Importamos Pandas, biblioteca utilizada en Python para la analítica de datos
import matplotlib.pyplot as plt # Manejo de gráficos
from google.colab import files # Manejo de archivos

---
# 3. Desarrollo

### Código en C y compilación

In [ ]:
code = """
#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP
#include <cmath>     // Impresión de números por terminal
#include <fstream>   // Para elevar a potencias y hacer raíces

// --- Macros para medir el tiempo
static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI(h)      \
   gettimeofday(&tv, NULL);   \
   dHashTiempoHistory[h] = tv.tv_sec + tv.tv_usec / 1000000.0;
#define TIEMPO_FIN(h)      \
   gettimeofday(&tv, NULL);   \
   dHashTiempoHistory[h] = ((tv.tv_sec + tv.tv_usec / 1000000.0) - dHashTiempoHistory[h]) * 1000; // milisegundos
#define TIEMPO_GET(h) dHashTiempoHistory[ h ]

#define HTH_TOTAL       1
#define HTH_BASILEA_SEC 2
#define HTH_BASILEA_OMP 3
// --- Fin macros para medir el tiempo


int main(int argc, char* argv[]) { 
    long i;
    double sum, pi_s, pi_omp;

    // Capturamos el tiempo inicial
    TIEMPO_INI(HTH_TOTAL)

    // Validamos parámetros y los obtenemos
    if(argc != 2){
        std::cerr<< "Error en los parámetros: debe indicar únicamente la cantidad de términos a sumar." << std::endl;
        exit(-1);
    }
    long cant_terminos = atoi(argv[1]);
    if(cant_terminos <= 0){
      std::cerr<< "Error en los parámetros: la cantidad de términos a sumar debe ser un número mayor o igual a 1." << std::endl;
      exit(-1);
    }
    
    // --- Problema de Basilea de forma secuencial 
    
    // Capturamos el tiempo inicial de la forma secuencial
    TIEMPO_INI(HTH_BASILEA_SEC)

    // Calculamos Pi
    sum = 0;
    for (i = 0; i < cant_terminos; i++){
        sum += 1/pow(i + 1, 2);   
    }
    pi_s = sqrt(6 * sum);

    // Capturamos el tiempo total de la forma secuencial
    TIEMPO_FIN(HTH_BASILEA_SEC)

    // --- Fin Problema de Basilea de forma secuencial
    
    // --- Problema de Basilea con OpenMP 

    // Capturamos el tiempo inicial con OpenMP
    TIEMPO_INI(HTH_BASILEA_OMP)

    // Calculamos Pi
    sum = 0;
    #pragma omp parallel for reduction(+: sum)
    for (i = 0; i < cant_terminos; i++){
        sum += 1/pow(i + 1, 2);   
    }
    pi_omp = sqrt(6 * sum); 
    
    // Capturamos el tiempo total con OpenMP
    TIEMPO_FIN(HTH_BASILEA_OMP)

    // Capturamos el tiempo total del ejercicio
    TIEMPO_FIN( HTH_TOTAL )

    std::cout<<std::endl;
    std::cout<<"--- Valores de Pi ---" <<std::endl;
    std::cout.precision(13);
    std::cout<<"Pi Secuencial: "<<pi_s<<" [ms]"<<std::endl;
    std::cout.precision(13);
    std::cout<<"Pi OpenMP    : "<<pi_omp<<" [ms]"<<std::endl;
    std::cout<<std::endl;

    std::cout<<"--- Métricas reales ---"<<std::endl;
    std::cout<<"Tiempo total     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
    std::cout<<"Tiempo secuencial: "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" [ms]"<<std::endl;
    std::cout<<"Tiempo OpenMP    : "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" [ms]"<<std::endl;
    std::cout<<"SpeedUp          : (tiempo secuencial/tiempo paralelo): "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" / "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)/TIEMPO_GET(HTH_BASILEA_OMP)<<std::endl;
    std::cout<<"Eficiencia       : SpeedUp / Nro. procesadores        : "<<TIEMPO_GET(HTH_BASILEA_SEC)/TIEMPO_GET(HTH_BASILEA_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP))<<std::endl;
    std::cout<<"Coste sec        : Nro. procesadores * tiempo         : "<<1<<" * "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)<<std::endl;
    std::cout<<"Coste OMP        : Nro. procesadores * tiempo         : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP)<<std::endl;
    std::cout<<"Funcion overhead : Coste OMP - tiempo secuencial      : "<<omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP)<<" - "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP))-TIEMPO_GET(HTH_BASILEA_SEC)<<std::endl;
    std::cout<<std::endl;

    std::cout<<"--- Métricas ideales ---"<<std::endl;
    TIEMPO_GET(HTH_BASILEA_OMP) = TIEMPO_GET(HTH_BASILEA_SEC) / 2;
    std::cout<<"Tiempo secuencial: "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" [ms]"<<std::endl;
    std::cout<<"Tiempo OpenMP    : "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" [ms]"<<std::endl;
    std::cout<<"SpeedUp          : (tiempo secuencial / tiempo paralelo): "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" / "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)/TIEMPO_GET(HTH_BASILEA_OMP)<<std::endl;
    std::cout<<"Eficiencia       : SpeedUp / Nro. procesadores          : "<<TIEMPO_GET(HTH_BASILEA_SEC)/TIEMPO_GET(HTH_BASILEA_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP))<<std::endl;
    std::cout<<"Coste sec        : Nro. procesadores * tiempo           : "<<1<<" * "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)<<std::endl;
    std::cout<<"Coste OMP        : Nro. procesadores * tiempo           : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP)<<std::endl;
    std::cout<<"Funcion overhead : Coste OMP - tiempo secuencial        : "<<omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP)<<" - "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP))-TIEMPO_GET(HTH_BASILEA_SEC)<<std::endl;
}
"""
# Verificamos que no haya inconvenientes en la generación del archivo o ejecutable
try:
  text_file = open("basilea.cpp", "w")
  text_file.write(code)
  text_file.close()
  !g++ -o basilea -fopenmp basilea.cpp
  print("¡Compilación exitosa!")
except:
  print("Ups! Algo salió mal en la generación del archivo o ejecutable.")

### Ejecución del algoritmo

In [ ]:
# Variable de entorno utilizada por OpenMP para la cantidad de threads
%env OMP_NUM_THREADS=16
# Ejecutamos
!./basilea 500000

### Medidas de prestaciones en algoritmos paralelos
Las tecnicas de HPC buscan reducir los tiempos de ejecución, el tiempo como media, no alcanza. Dos algoritmos pueden ejecutar en el mismo tiempo, pero uno de ellos usa menos procesadores [7][8]. 


#### **SpeedUp**
Referencia a la ganacia de velocidad que se consigue con un algoritmo paralelo, al resolver el mismo problema con respecto al algoritmo secuencial.

<center>$ SpeedUp = TiempoSecuencial  /  TiempoParalelo $</center>



#### **Eficiencia**
La eficiencia normaliza el valor del SpeedUp, al dividirlo por la cantidad de procesadores que se utilizaron para alcanzar la ganacia en velocidad. Dando la idea de la porción de tiempo que los procesadores se dedican al trabajo útil.

<center>$ Eficiencia = SpeedUp  /Nro. procesadores$</center>


#### **Coste**
El coste de un algoritmo paralelo representa el tiempo realizado por todo el sistema en la resoluciòn del problema.

<center>$ coste = Nro. procesadores * Tiempo algoritmo$</center>


#### **Función Overhead**
Es la diferencia entre el Coste y el tiempo secuencial. Mientras mayor es la función overhead, peor es el comportamiento del algoritmo paralelo.
<center>$ Overhead = Coste  /  TiempoSecuencial $</center>


### ¿Cómo se aproxima el cálculo de $\pi$ a medida que agregamos términos?

Vamos a sacar provecho de una biblioteca de Python especializada en análitica de datos llamada **Pandas** [9]. Lo que haremos es obtener los resultados aproximados de $\pi$ obtenidos a través del algoritmo y ver cómo evoluciona el valor a medida que la cantidad de términos aumenta.

Por último, podremos descargarnos los resultados en una planilla de Excel a través de una biblioteca propia de Colab llamada **files** [10].

#### Generación de gráfico a partir de dataframe

In [ ]:
datos = {
    'Cantidad de términos': [
      100, 500, 
      1000, 5000, 
      10000, 50000, 
      100000, 500000, 
      1000000, 5000000, 
      10000000, 50000000,
      100000000, 500000000
    ],
    'Pi': [
      3.132076531809, 3.139684123139,
      3.140638056206, 3.141401680951,
      3.141497163947, 3.14157355513,
      3.141583104326, 3.141590743732,
      3.141591698661, 3.141592462604,
      3.141592558097, 3.141592634491,
      3.141592644041, 3.141592651706
    ]
}
try:
  df = pd.DataFrame(datos, columns=['Cantidad de términos','Pi'])
  df.plot(
      x = 'Cantidad de términos', y = 'Pi', xlim=(100,1000000000), loglog = True,
      figsize=(10, 5), grid = True, color = "red",
      title = 'Aproximación de Pi según cantidad de términos de Basilea'
  )
except:
  print("No pudo generarse el dataframe, ¿realizó el armado previo del ambiente?")

Podemos ver que a partir de **100 mil términos de basilea** la aproximación de $\pi$ parece estabilizarse. Por supuesto, el nivel de precisión va a ser adecuado o no según nuestra aplicación. Por ejemplo, si tenemos que enviar un satélite al espacio, el cálculo de la órbita para la trayectoria necesitará muchos decimales de presición.

#### Podemos descargarnos los resultados en una planilla de Excel

In [ ]:
try:
  excel_descarga = 'pi_basilea.xls'
  df.to_excel(excel_writer = excel_descarga)
  files.download(excel_descarga)
except:
  print("No pudo generarse la planilla de Excel, ¿generó previamente el dataframe?")

# 4. Tabla de pasos

Ámbito|Función|Detalle
---|---|---
Colab|import|Importa los módulos para funcionar.
C|include|Incluye las bibliotecas para el código C.
C|define|Define macros y constantes.
C|TIEMPO_INI|Captura el tiempo inicial del ejercicio.
C|if(argc)|Valida la cantidad de parámetros.
C|atoi|Pasamos la cantidad de términos a long.
C|if(...)|Verificamos que la cantidad de términos sea válida.
C|TIEMPO_INI|Captura el tiempo inicial del modo secuencial.
C|for(...)|Aplica el algortimo de forma secuencial.
C|TIEMPO_FIN|Captura el tiempo total del modo secuencial.
C|TIEMPO_INI|Captura el tiempo inicial con OpenMP.
C|pragma omp parallel for|Indica la parelización del for. 
C|for(...)|Aplica el algortimo con OpenMP.
C|reduction(+: sum)|Indica a OpenMP que se compartirá la variable sum y se reducirá a través de la operación '+'.
C|TIEMPO_FIN|Captura el tiempo total de OpenMP y de todo el ejercicio.
C|cout|Imprimimos resultados.
Colab|open|Abrimos archivo para escribir.
Colab|write|Escribimos el archivo con el código C.
Colab|close|Cerramos el archivo.
Colab|g++ ... -fopenmp...|Compilamos indicándole al compilador que hay directivas OpenMP.
Colab|%env OMP_NUM_THREADS|Seteamos la cantidad de threadas de OpenMP a través de la variable de entorno.
Colab|basilea...|Ejecutamos el código.
Colab|datos{...}|Definimos una estructura con los datos obtenidos.
Colab|pd.DataFrame|Generamos el dataframe con los datos.
Colab|df.plot|Generamos el gráfico a partir del dataframe.
Colab|df.to_excel|Generamos el excel a partir del dataframe.
Colab|file.download|Descargamos el archivo de excel a nuestro dispositivo.

# 5. Conclusiones

Se realizaron las siguientes pruebas:

Cant. de términos|Tiempo secuencial [ms]|Tiempo OpenMP [ms]| Mejora performance [%] 
---|---|---|---
100|0.003|1.128|N/A
500|0.006|0.711|N/A
1K|0.010|0.745|N/A
5K|0.072|1.432|N/A
10K|0.162|4.169|N/A
50K|0.777|1.565|N/A
100K|1.267|1.899|N/A
500K|9.341|6.467|30.77 %
1M|10.516|9.779|7 %
5M|54.245|45.366|16.39 %
10M|103.356|92.069|10.92 %
50M|507.103|439.947|13.24 %
100M|1002.540|841.186|16.09 %
500M|4969.376|4205.069|15.38 %

Podemos considerar varios aspectos:

*   Este es otro problema cuya **complejidad es lineal** a la cantidad de términos, al menos, en términos temporales.

*   El **uso de series** para el cálculo de $\pi$ es una de las mejores alternativas frente a otros como el Método de Arquímedes [11] o el Método Montecarlo [12] (aunque este último tiene muchas aplicaciones, como el cálculo de areas complejas o la economía [13]). Sin embargo, La serie del Problema de Basilea **no es necesariamente la más eficiente de todas**.

Consideramos, entonces, que a partir del cálculo de **500 mil términos** empieza a ser justificable la idea de **paralelizar** el cálculo de $\pi$, menor a esa cantidad incluso perderíamos eficiencia debido al overhead y la sincronización de los hilos.


Una manera de mejorar el ejercicio podría ser que a, a partir del código C, se hagan pruebas con diveras cantidades de términos de basilea para el cálculo de $\pi$ y que, al final, se genere un CSV el cual podremos manipular directamente con Pandas. 

---
# 6. Bibliografía

[1] Euler y El Problema de Basilea: [PDF](http://www2.caminos.upm.es/Departamentos/matematicas/revistapm/revista_impresa/vol_V_num_1/his_mat_euler_basilea.pdf)

[2] Introducción a OpenMP: [OpenMp](https://www.openmp.org/wp-content/uploads/omp-hands-on-SC08.pdf)

[3] Directivas OpenMP: [Página web](https://bisqwit.iki.fi/story/howto/openmp/#DistributeParallelForConstruct)

[4] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[5] Sintaxis Markdown Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[6] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[7] F. Almeida, D. Gimenéz, A. Vidal - Introducción a la programación paralela - 2008 - Editorial Parafino.

[8] D. Jiménez-González - Introducción a las arquitecturas paralelas. [PDF](http://so-unlam.com.ar/material-clase/HPC/Arquitecturas_de_computadores_avanzadas_(Modulo_1).pdf)

[9] Biblioteca Pandas. [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)

[10] Manejo de archivos desde Colab. [Colab](https://colab.research.google.com/notebooks/io.ipynb)

[11] Método de Arquímedes. [DocIRS](https://www.docirs.cl/calculo_pi.htm)

[12] Método Montecarlo. [Geogebra](https://www.geogebra.org/m/cF7RwK3H)

[13] Montecarlo en la Economía [Master en Finanzas](https://www.master-finanzas-cuantitativas.com/metodos-de-montecarlo/)